In [1]:
# The code was removed by Watson Studio for sharing.

In [ ]:
import os
from datetime import datetime, timedelta
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import time

from MLOps_CPD.jenkins_jobs.pr_jenkins_job import JobRunner

import ibm_watson_openscale
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator, IAMAuthenticator
from ibm_watson_machine_learning import APIClient as wmlapiclient
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *


In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
CLOUD_API_KEY = os.getenv('CLOUD_API_KEY')
SPACE_ID = os.getenv("SPACE_ID")
SERVICE_PROVIDER_ID = os.getenv("SERVICE_PROVIDER_ID")
SUBSCRIPTION_ID = os.getenv("SUBSCRIPTION_ID")
job_name = os.getenv("JOB_NAME")
project_id = os.environ['PROJECT_ID']

mlops_cos_credentials = {'API_KEY':API_KEY_MLOPS,
                          'CRN':CRN_MLOPS,
                          'AUTH_ENDPOINT':AUTH_ENDPOINT,
                           'ENDPOINT_URL':ENDPOINT_URL_MLOPS,
                           'BUCKET':BUCKET_MLOPS
                          }


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
wos_client = APIClient(authenticator=authenticator, service_instance_id=DATA_MART_ID)

## OpenScale Utils

In [ ]:
from operator import itemgetter

def get_monitor_instance_id_by_name(subscription_id,monitor):
    for item in wos_client.monitor_instances.list().result.to_dict()['monitor_instances']:
        if(item['entity']['monitor_definition_id']) ==monitor and item['entity']['target']['target_id']==subscription_id:
            return item['metadata']['id']
        
        
def get_monitor_metrics(wos_client, monitor_instance_id):
    start_time = datetime.now() - timedelta(days=7)
    end_time = datetime.now()
    runs = wos_client.monitor_instances.runs.list(
        monitor_instance_id=monitor_instance_id
    ).result.runs
    measurements = ibm_watson_openscale.base_classes.watson_open_scale_v2.Measurements(
        watson_open_scale=wos_client.monitor_instances._ai_client
    )
    for run in runs:
        run_id = run.to_dict()["metadata"]["id"]
        response = measurements.list(
            monitor_instance_id=monitor_instance_id,
            start=start_time,
            end=end_time,
            run_id=run_id,
        )
        #print(response.result.to_dict())
        metrics = response.result.to_dict()["measurements"][0]["entity"]["values"][0][
            "metrics"
        ]
        return metrics
    
    
def get_measurement_id(wos_client,monitor_instance_id):
    start_time = datetime.now() - timedelta(days=7)
    end_time = datetime.now()
    runs = wos_client.monitor_instances.runs.list(monitor_instance_id=monitor_instance_id).result.runs
    final =[]
    for run in runs:
        each = run.to_dict()["metadata"]["id"]
        response= wos_client.monitor_instances.measurements.list(monitor_instance_id=m_id,start=start_time,end=end_time,run_id=each)
        result = response.result.to_dict()
        final.append(result['measurements'][0]['metadata'])
        #print(result['measurements'][0]['metadata'])
    
    my_asset_list = sorted(final, key=itemgetter("created_at"), reverse=True)
    return my_asset_list[0]['id']


def get_annotations(measurement_response):
    sources_data = measurement_response.entity.sources[0].data
    
    annotations = {}
    
    if "copy_of" in sources_data:
        original_measurement_id = sources_data["copy_of"]["measurement_id"]
    else:
        original_measurement_id = measurement_response.metadata.id
        
    if "drifted_transactions" in sources_data:
        annotations["drifted_transactions"] = sorted(["drift_{}_cluster_{}".format(original_measurement_id, cluster["id"]) for cluster in sources_data["drifted_transactions"]["clusters"]])
        
    if "data_drifted_transactions" in sources_data:
        annotations["data_drifted_transactions"] = sorted(["ddrift_{}_cluster_{}".format(original_measurement_id, cluster["id"]) for cluster in sources_data["data_drifted_transactions"]["clusters"]])
    
    if "model_data_drifted_transactions" in sources_data:
        annotations["model_data_drifted_transactions"] = sorted(["mddrift_{}_cluster_{}".format(original_measurement_id, cluster["id"]) for cluster in sources_data["drifted_transactions"]["clusters"]])
    
    return annotations

def get_dataset_id(name,subscription_id):
    import time
    time.sleep(5)
    data_set_id = None
    if name == "PAYLOAD_LOGGING":
        types = DataSetTypes.PAYLOAD_LOGGING
    elif name == "FEEDBACK":
        types = DataSetTypes.FEEDBACK 
    data_set_id = wos_client.data_sets.list(type=types, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
    if data_set_id is None:
        print("Payload data set not found. Please check subscription status.")
    
    return data_set_id

def read_data_from_mlops_cos(key):
    def __iter__(self): return 0
    MLOPS_DATA_STORE_client = ibm_boto3.client(
        service_name='s3',
        ibm_api_key_id=API_KEY_MLOPS,
        ibm_service_instance_id=CRN_MLOPS,
        ibm_auth_endpoint=AUTH_ENDPOINT,
        config=Config(signature_version='oauth'),
        endpoint_url=ENDPOINT_URL_MLOPS)

    body = MLOPS_DATA_STORE_client.get_object(Bucket='mlops-asset', Key=key)['Body']
    # add missing __iter__ method, so pandas accepts body as file-like object
    if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

    gcf_df = pd.read_csv(body)
    return gcf_df
def save_data_in_cos(df,filename,key,credentials,pipe=False):
    """

    Save Data in IBM Cloud Object Storage

    """
    try:
        
        if pipe:
            with open (filename,'wb') as f:
                pickle.dump(pipeline,f)
        else:
            df.to_csv(filename,index=False)
        mlops_res = ibm_boto3.resource(
            service_name='s3',
            ibm_api_key_id=credentials['API_KEY'],
            ibm_auth_endpoint=credentials['AUTH_ENDPOINT'],
            config=Config(signature_version='oauth'),
            endpoint_url=credentials['ENDPOINT_URL'])

        mlops_res.Bucket(credentials['BUCKET']).upload_file(filename,key)
        print(f"File {filename} uploaded successfully")
    except Exception as e:
        print(e)
        print("File upload for {filename} failed")
        
        
def push_driver(apikey, project_id, job_name):
    jobrun = JobRunner(apikey, project_id)
    job_id = jobrun.retrieve_job_id(name=job_name)
    print(f" Job ID is -->{job_id}")

    ##Start the Pipeline Run

    jobrun.run_pipeline_job(job_id)

    job_runs = jobrun.get_run_ids(job_id)

    latest_job_run_id = list(job_runs.keys())[0]

    print(f"Latest Job RUN ID is -->{latest_job_run_id}")

    try:
        while True:
            result = jobrun.get_runs_by_runid(job_id=job_id, run_id=latest_job_run_id)
            state = result["entity"]["job_run"]["state"]
            if state == "Completed":
                break
            elif state == "Failed":
                raise Exception("Job Run Failed")
            print(state)
            time.sleep(120)
    except Exception as e:
        print(e)

    print(
        f"Job {job_id} with latest run {latest_job_run_id} done with status--> {state}"
    )
    # pprint(jobrun.get_runs_by_runid(job_id=job_id, run_id=latest_job_id))


## Get Measurement IDs and Extract Measurements

In [ ]:
m_id = get_monitor_instance_id_by_name(SUBSCRIPTION_ID,'drift')

In [ ]:
measurement_id = get_measurement_id(wos_client,m_id)

In [ ]:
get_monitor_metrics(wos_client=wos_client,monitor_instance_id=m_id)

In [ ]:
measurement = wos_client.monitor_instances.measurements.get(measurement_id=measurement_id, monitor_instance_id=m_id).result

In [ ]:
payload_data_set_id = get_dataset_id(name='PAYLOAD_LOGGING',subscription_id=SUBSCRIPTION_ID)
feedback_data_set_id = get_dataset_id(name='FEEDBACK',subscription_id=SUBSCRIPTION_ID)

## Extract Drifted Records

In [ ]:
drifted_df = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, annotations=get_annotations(measurement)["data_drifted_transactions"],output_type='pandas').result
drifted_df.shape

In [ ]:
discarded = ['asset_revision', 'debiased_probability','scoring_timestamp','scoring_id','probability','debiased_prediction','deployment_id']
drifted_df= drifted_df.drop(discarded,axis=1)

In [ ]:
final_transactions = drifted_df[drifted_df['prediction_probability']>0.75]
final_transactions= final_transactions.rename(columns={'prediction':'Risk'})
final_transactions['Risk']= final_transactions['Risk'].map({1:'Risk',0:'No Risk'})
final_transactions = final_transactions.drop('prediction_probability',axis=1)
final_transactions['ForeignWorker'] = "no"
final_transactions['InstallmentPlans'] = "none"

In [ ]:
final_transactions.shape

In [ ]:
original_training_data = read_data_from_mlops_cos("german_credit_risk_hist.csv")
original_training_data.shape

## Combine the relevant high confidence records to the Training Data 

In [ ]:
new_df = pd.concat([final_transactions,original_training_data]).reset_index(drop=True)

In [ ]:
#save_data_in_cos(new_df,"german_credit_risk.csv","german_credit_risk.csv",mlops_cos_credentials)

## Kick Off the WS Pipeline. Supply the name of the job to run

In [ ]:
#push_driver(apikey=CLOUD_API_KEY,project_id=project_id,job_name=job_name)